In [1]:
%load_ext autoreload
%autoreload 2
import math

from projectq import MainEngine
from projectq.ops import H, Z, X, Measure, All
from projectq.meta import Loop, Compute, Uncompute, Control

from qcircuit import QCircuit, Classic, Qubit, Qureg


In [2]:
n_qubits = 3
circuitInit = QCircuit(oracle_out=Qubit, x=(n_qubits, ))
circuitInit

In [3]:
circuitLoop = QCircuit(oracle_out=Qubit, x=(n_qubits, ))
circuitLoop

In [4]:
oracle = QCircuit(oracle_out=Qubit, x=(n_qubits, ))
oracle

In [5]:
import math

from projectq import MainEngine
from projectq.ops import H, Z, X, Measure, All
from projectq.meta import Loop, Compute, Uncompute, Control


def run_grover(eng, n):
    """
    Runs Grover's algorithm on n qubit using the provided quantum oracle.

    Args:
        eng (MainEngine): Main compiler engine to run Grover on.
        n (int): Number of bits in the solution.
        oracle (function): Function accepting the engine, an n-qubit register,
            and an output qubit which is flipped by the oracle for the correct
            bit string.

    Returns:
        solution (list<int>): Solution bit-string.
    """
    x = eng.allocate_qureg(n)
    oracle_out = eng.allocate_qubit()

    # number of iterations we have to run:
    num_it = int(math.pi/4.*math.sqrt(1 << n))

    circuitInit(eng, oracle_out, x)
    
    # run num_it iterations
    with Loop(eng, num_it):
        # oracle adds a (-1)-phase to the solution
        oracle(eng, oracle_out, x)

        # reflection across uniform superposition

        circuitLoop(eng, oracle_out, x)

    All(Measure) | x
    Measure | oracle_out

    eng.flush()
    # return result
    return [int(qubit) for qubit in x]


if __name__ == "__main__":
    eng = MainEngine()  # use default compiler engine
    # run Grover search to find a 7-bit solution
    print(run_grover(eng, n_qubits))

(Note: This is the (slow) Python simulator.)
oracle_out
x
2
Debug:  H  |  x
Debug:  H  |  x
Debug:  H  |  x
Debug:  X  |  oracle_out
Debug:  H  |  oracle_out
oracle_out
x
2
Debug:  X  |  x
Debug:  X  |  x
Debug:  X  with Control( ['1', '2', '3'] )
Debug:  X  |  x
Debug:  X  |  x
oracle_out
x
2
Debug:  H  |  x
Debug:  H  |  x
Debug:  H  |  x
Debug:  X  |  x
Debug:  X  |  x
Debug:  X  |  x
Debug:  Z  with Control( ['2', '1'] )
Debug:  X  |  x
Debug:  X  |  x
Debug:  X  |  x
Debug:  H  |  x
Debug:  H  |  x
Debug:  H  |  x
[0, 1, 0]
